In [13]:
from pathlib import Path
import os
os.chdir('/Users/federicoferoggio/Documents/vs_code/latent-communication')

import torch
import hydra
from utils.dataloaders.dataloader_mnist_single import DataLoaderMNIST
from torchvision.datasets import MNIST
from torchvision.transforms import transforms

from models.definitions.PocketAutoencoder import PocketAutoencoder

from utils.model import load_model, get_transformations
from utils.sampler import simple_sampler
from utils.visualization import visualize_results

from optimization.optimizer import LinearFitting
from optimization.optimizer import AffineFitting

DEVICE = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
augmentations = [transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]

class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

In [8]:
order_check = []

pth_files = [file for file in os.listdir('models/checkpoints/SMALLAE/MNIST/') if os.path.isfile(os.path.join('models/checkpoints/SMALLAE/MNIST/', file)) and file.endswith('.pth')]

for file in pth_files:
    model_order = []
    latent_space_dict = {}
    model1 = PocketAutoencoder()
    model1.load_state_dict(torch.load(f'models/checkpoints/SMALLAE/MNIST/{file}'))
    model1.to(DEVICE)
    model1.eval()
    
    DataLoaders = DataLoaderMNIST
    indices = range(40000)  # Assuming you want to use the first 40,000 images
    data_loader = DataLoaderMNIST(128, augmentations, indices=indices)
    test_loader = data_loader.get_test_loader()

    # Get all images from test_loader and convert them to latent space
    for batch_idx, (images, labels) in enumerate(test_loader):
        images = images.to(DEVICE)
        with torch.no_grad():
            latent_space = model1.get_latent_space(images)

        # Convert tensor to numpy array
        latent_space_np = latent_space.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()
        
        # Store in dictionary
        for idx in range(len(images)):
            latent_space_dict[batch_idx * 128 + idx] = (latent_space_np[idx], labels_np[idx])
            model_order.append(labels_np[idx])
    order_check.append(model_order)
    torch.save(latent_space_dict, 'models/checkpoints/SMALLAE/MNIST/LATENTS/' + str(file).replace('.pth', '_latent_space.pth'))

## Check they are in the same order
for i in range(len(order_check)-1):
    if order_check[i] != order_check[i+1]:
        print('Order is not the same')
        break

In [12]:
def create_mapping(cfg, latents1, latents2):
    if cfg.mapping == 'Linear':
        from optimization.optimizer import LinearFitting
        mapping = LinearFitting(latents1, latents2, lamda=cfg.lamda)
    elif cfg.mapping == 'Affine':
        from optimization.optimizer import AffineFitting
        mapping = AffineFitting(latents1, latents2, lamda=cfg.lamda)
    elif cfg.mapping == 'NeuralNetwork':
        from optimization.optimizer import NeuralNetworkFitting
        mapping = NeuralNetworkFitting(latents1, latents2, hidden_dim=cfg.hidden_size, lamda=cfg.lamda, learning_rate=cfg.learning_rate, epochs=cfg.epochs)
    else:
        raise ValueError("Invalid experiment name")
    return mapping

In [17]:

for file1 in os.listdir('models/checkpoints/SMALLAE/MNIST/LATENTS/'):
    if file1.endswith('.pth'):
        latents1 = torch.load('models/checkpoints/SMALLAE/MNIST/LATENTS/' + file1)
        for file2 in os.listdir('models/checkpoints/SMALLAE/MNIST/LATENTS/'):
            if file1 != file2 and file2.endswith('.pth'):
                latents2 = torch.load('models/checkpoints/SMALLAE/MNIST/LATENTS/' + file2)

                parameters = {
                    "num_samples": 49,
                    "mapping": "Linear",
                    "lamda": 0.01,
                }
                latent1_pl = torch.stack([torch.tensor(value[0]) for value in latents1.values()])
                latent2_pl = torch.stack([torch.tensor(value[0]) for value in latents2.values()])
                cfg = Config(**parameters)
                mapping = create_mapping(cfg, latent1_pl, latent2_pl)
                mapping.fit()
                storage_path = f'results/transformations/SMALLAE/{cfg.mapping}_{file1[:-len("_latent_space.pth")]}_{file2[:-len("_latent_space.pth")]}_{cfg.num_samples}'
                Path(storage_path).parent.mkdir(parents=True, exist_ok=True)
                mapping.save_results(storage_path)
                latents1_trafo = mapping.transform(latent1_pl)

                # for n in range(10):
                #     desired_class = n  # Specify the class you want to filter
                #     filtered_samples = []

                #     # Filter samples from the test loader based on the desired class
                #     for data, label in test_loader:
                #         indices = torch.nonzero(label == desired_class, as_tuple=False)
                #         if indices.numel() > 0:
                #             for idx in indices:
                #                 filtered_samples.append((data[idx], label[idx]))

Defining the problem


/Users/federicoferoggio/Documents/vs_code/latent-communication/.zeroshot/lib/python3.9/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


Solving the problem


/Users/federicoferoggio/Documents/vs_code/latent-communication/.zeroshot/lib/python3.9/site-packages/cvxpy/problems/problem.py:164: UserWarning: Constraint #0 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
